In [2]:
import os
# from dotenv import load_dotenv
from genai.credentials import Credentials
credentials = Credentials(api_key="pak-8prwoIymlhiNFNhxydsvEL8pl2VnQi_84sy03fvyLC0", api_endpoint="https://bam-api.res.ibm.com")

In [30]:
import os

os.environ["GENAI_KEY"] = "pak-8prwoIymlhiNFNhxydsvEL8pl2VnQi_84sy03fvyLC0"
os.environ["GENAI_API"] = "https://bam-api.res.ibm.com/"

from genai.credentials import Credentials
from genai.client import Client
credentials = Credentials.from_env()
client = Client(credentials=Credentials.from_env())

In [3]:
from langchain.agents import AgentExecutor
from langchain.tools import Tool
from genai.client import Client
from genai.credentials import Credentials
from genai.schema import (
    DecodingMethod,
    HumanMessage,
    ModerationHAP,
    ModerationHAPInput,
    ModerationHAPOutput,
    ModerationParameters,
    SystemMessage,
    TextGenerationParameters,
)

In [4]:
parameters = TextGenerationParameters(
    decoding_method=DecodingMethod.SAMPLE,
    max_new_tokens=128,
    min_new_tokens=30,
    temperature=0.7,
    top_k=50,
    top_p=1
)

In [5]:
model_id = "meta-llama/llama-2-70b-chat"
prompt = "What is NLP and how it has evolved over the years?"

In [6]:
response = client.text.chat.create(
    model_id=model_id,
    messages=[
        SystemMessage(
            content="""You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
any sense, or is not factually coherent, explain why instead of answering something incorrectly.
If you don't know the answer to a question, please don't share false information.
""",
        ),
        HumanMessage(content=prompt),
    ],
    parameters=parameters,
)
conversation_id = response.conversation_id
print(f"Conversation ID: {conversation_id}")
print(f"Request: {prompt}")
print(f"Response: {response.results[0].generated_text}")
prompt = "How can I start?"
response = client.text.chat.create(
    messages=[HumanMessage(content=prompt)],
    moderations=ModerationParameters(
        hap=ModerationHAP(
            input=ModerationHAPInput(enabled=True, threshold=0.8),
            output=ModerationHAPOutput(enabled=True, threshold=0.8),
        )
    ),
    conversation_id=conversation_id,
    use_conversation_parameters=True,
)
print(f"Request: {prompt}")
print(f"Response: {response.results[0].generated_text}")

Conversation ID: 5f9788df-26d0-4280-bece-5d0ab464cd78
Request: What is NLP and how it has evolved over the years?
Response:   NLP stands for Natural Language Processing, which is a subfield of artificial intelligence (AI) that deals with the interaction between computers and humans in natural language. It involves the development of algorithms and statistical models that enable computers to process, understand, and generate natural language data.

NLP has evolved significantly over the years, driven by advances in machine learning, computational power, and the availability of large datasets. Here's a brief overview of the major milestones in the evolution of NLP:

1. Early Beginnings (1950s-1960s
Request: How can I start?
Response:   To get started with NLP, you can follow these steps:

1. Learn the basics of programming: NLP involves writing code to process and analyze natural language data. Therefore, it's essential to have a good understanding of programming concepts, data structure

In [7]:
from typing import Optional

from dotenv import load_dotenv
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import render_text_description_and_args
from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import BaseTool

from genai import Client, Credentials
from genai.extensions.langchain import LangChainChatInterface
from genai.schema import TextGenerationParameters

load_dotenv()


class WordLengthTool(BaseTool):
    name = "GetWordLength"
    description = "Returns the length of a word."

    def _run(self, word: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> int:
        return len(word)


tools: list[BaseTool] = [WordLengthTool()]

system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""
human_prompt = """{input}
{agent_scratchpad}
(reminder to respond in a JSON blob no matter what)"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

client = Client(credentials=Credentials.from_env())
llm = LangChainChatInterface(
    client=client,
    model_id="meta-llama/llama-2-70b-chat",
    parameters=TextGenerationParameters(
        max_new_tokens=250, min_new_tokens=20, temperature=0, stop_sequences=["\nObservation"]
    ),
)
prompt = prompt.partial(
    # get tools with their descriptions and args in plain text
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

memory = ConversationBufferMemory()

agent = (
    RunnablePassthrough.assign(
        # format the agent's scratchpad to a string
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        # pass the memory as the chat history
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory)

agent_executor.invoke({"input": "How many letters are in the word educa?"})
agent_executor.invoke({"input": "That's not a real word, can you tell me a valid word?"})



> Entering new AgentExecutor chain...
  Action:
```
{
  "action": "GetWordLength",
  "action_input": "educA"
}
```
Observation5  Action:
```
{
  "action": "Final Answer",
  "action_input": "The word 'educA' has 5 letters."
}
```
Observation

> Finished chain.


> Entering new AgentExecutor chain...
  {
"action": "GetWordLength",
"action_input": "educA"
}

Observation5  Action:
```
{
  "action": "GetWordLength",
  "action_input": "education"
}
```
Observation9  Action:
```
{
  "action": "Final Answer",
  "action_input": "The word 'education' has 9 letters."
}
```
Observation

> Finished chain.


{'input': "That's not a real word, can you tell me a valid word?",
 'history': "Human: How many letters are in the word educa?\nAI: The word 'educA' has 5 letters.",
 'output': "The word 'education' has 9 letters."}

In [52]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)

# make sure you have a .env file under genai root with
# GENAI_KEY=<your-genai-key>
# GENAI_API=<genai-api-endpoint> (optional) DEFAULT_API = "https://bam-api.res.ibm.com"
load_dotenv()


def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"


llm = LangChainChatInterface(
    client=Client(credentials=Credentials.from_env()),
    model_id="meta-llama/llama-2-70b-chat",
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=100,
        min_new_tokens=10,
        temperature=0.5,
        top_k=50,
        top_p=1,
        return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
    ),
    moderations=ModerationParameters(
        # Threshold is set to very low level to flag everything (testing purposes)
        # or set to True to enable HAP with default settings
        hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
    ),
)

print(heading("Start conversation with langchain"))
prompt = "Describe what is Python in one sentence."
print(f"Request: {prompt}")
result = llm.generate(
    messages=[
        [
            SystemMessage(
                content="""You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
any sense, or is not factually coherent, explain why instead of answering something incorrectly.
If you don't know the answer to a question, please don't share false information.
""",
            ),
            HumanMessage(content=prompt),
        ]
    ],
)
conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
print(f"New conversation with ID '{conversation_id}' has been created!")
print(f"Response: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

print(heading("Continue conversation with langchain"))
prompt = "Show me some simple code example."
print(f"Request: {prompt}")
result = llm.generate(
    messages=[[HumanMessage(content=prompt)]], conversation_id=conversation_id, use_conversation_parameters=True
)
print(f"Response: {result.generations[0][0].text}")


====================== Start conversation with langchain =======================

Request: Describe what is Python in one sentence.
New conversation with ID '1032015e-a281-4b65-85c8-5c0e0cd068fb' has been created!
Response:   Python is a popular, versatile, and user-friendly programming language that is widely used for various purposes such as web development, data analysis, artificial intelligence, scientific computing, and more.
{'model_name': 'meta-llama/llama-2-70b-chat', 'token_usage': {'prompt_tokens': 150, 'completion_tokens': 41, 'total_tokens': 191}}
{'meta': {'conversation_id': '1032015e-a281-4b65-85c8-5c0e0cd068fb', 'created_at': datetime.datetime(2024, 5, 5, 17, 48, 18, 614000, tzinfo=TzInfo(UTC)), 'id': 'db02e8c2-1628-4a39-a8e0-a73558e03cda', 'model_id': 'meta-llama/llama-2-70b-chat'}, 'token_usage': {'prompt_tokens': 150, 'completion_tokens': 41, 'total_tokens': 191}, 'generated_token_count': 41, 'input_token_count': 150, 'input_tokens': [{'text': '<s>'}, {'text': '▁['},

### With memory

In [56]:
import time
session_id = str(int(time.time()))
session_id

'1714932570'

In [12]:
import uuid
session_id = str(uuid.uuid4())
session_id

'3b2d0e11-5055-4ad7-b17c-6626bb193fbb'

In [8]:
# from dotenv import load_dotenv
# from langchain_core.messages import HumanMessage, SystemMessage
# from genai import Client, Credentials
# from genai.extensions.langchain.chat_llm import LangChainChatInterface
# from genai.extensions.langchain import LangChainInterface
# from genai.schema import (
#     DecodingMethod,
#     ModerationHAP,
#     ModerationHAPInput,
#     ModerationParameters,
#     TextGenerationParameters,
#     TextGenerationReturnOptions,
# )
# from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_core.runnables.history import RunnableWithMessageHistory

# # Load environment variables
# load_dotenv()

# def heading(text: str) -> str:
#     """Helper function for centering text."""
#     return "\n" + f" {text} ".center(80, "=") + "\n"

# # Create the custom template prompt
# client = Client(credentials=Credentials.from_env())
# prompt_response = client.prompt.create(
#     name="Recipe Generator Prompt",
#     model_id="google/flan-t5-xl",
#     input="Make a short recipe for {{meal}} (use bullet points)",
# )

# # Create the LangChain Chat Interface with memory functionality
# message_history = ChatMessageHistory()
# agent_with_chat_history = RunnableWithMessageHistory(
#     LangChainChatInterface(
#         client=client,
#         model_id="ibm/granite-13b-instruct-v2",
#         prompt_id=prompt_response.result.id,
#         parameters=TextGenerationParameters(
#             decoding_method=DecodingMethod.SAMPLE,
#             max_new_tokens=100,
#             min_new_tokens=10,
#             temperature=0.5,
#             top_k=50,
#             top_p=1,
#             return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
#         ),
#         moderations=ModerationParameters(
#             hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
#         ),
#         data={"meal": "Lasagne"},
#     ),
#     lambda session_id: message_history,
#     input_messages_key="input",
#     history_messages_key="chat_history",
# )

# # Start a conversation with memory functionality
# print(heading("Start conversation with langchain"))
# prompt = "Describe what is Python in one sentence."
# print(f"Request: {prompt}")
# result = agent_with_chat_history.invoke(
#     [
#         SystemMessage(
#             content="""You are a helpful, respectful and honest assistant.
# Always answer as helpfully as possible, while being safe.
# Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
# Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
# any sense, or is not factually coherent, explain why instead of answering something incorrectly.
# If you don't know the answer to a question, please don't share false information.
# """,
#             user_id="assistant"
#         ),
#         HumanMessage(content=prompt, user_id="user")
#     ],
#     {"configurable": {"session_id": session_id}}
# )

# conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
# print(f"New conversation with ID '{conversation_id}' has been created!")
# print(f"Response: {result.generations[0][0].text}")
# print(result.llm_output)
# print(result.generations[0][0].generation_info)

# # Continue the conversation with memory functionality
# print(heading("Continue conversation with langchain"))
# prompt = "Show me some simple code example."
# print(f"Request: {prompt}")
# result = agent_with_chat_history.invoke(
#     {"input": [HumanMessage(content=prompt, user_id="user")]},
#     {"configurable": {"session_id": conversation_id}},
#     use_conversation_parameters=True,
# )
# print(f"Response: {result.generations[0][0].text}")

# # Delete the prompt if no longer needed
# client.prompt.delete(prompt_response.result.id)

In [8]:
# from dotenv import load_dotenv
# from langchain_core.messages import HumanMessage, SystemMessage
# from genai import Client, Credentials
# from genai.extensions.langchain.chat_llm import LangChainChatInterface
# from genai.extensions.langchain import LangChainInterface
# from genai.schema import (
#     DecodingMethod,
#     ModerationHAP,
#     ModerationHAPInput,
#     ModerationParameters,
#     TextGenerationParameters,
#     TextGenerationReturnOptions,
# )
# from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_core.runnables.history import RunnableWithMessageHistory

# # Load environment variables
# load_dotenv()

# def heading(text: str) -> str:
#     """Helper function for centering text."""
#     return "\n" + f" {text} ".center(80, "=") + "\n"

# # Create the custom template prompt
# client = Client(credentials=Credentials.from_env())
# prompt_response = client.prompt.create(
#     name="Recipe Generator Prompt",
#     model_id="google/flan-t5-xl",
#     input="Make a short recipe for {{meal}} (use bullet points)",
# )

# # Create the LangChain Chat Interface with memory functionality
# message_history = ChatMessageHistory()
# agent_with_chat_history = RunnableWithMessageHistory(
#     LangChainChatInterface(
#         client=client,
#         model_id="ibm/granite-13b-instruct-v2",
#         prompt_id=prompt_response.result.id,
#         parameters=TextGenerationParameters(
#             decoding_method=DecodingMethod.SAMPLE,
#             max_new_tokens=100,
#             min_new_tokens=10,
#             temperature=0.5,
#             top_k=50,
#             top_p=1,
#             return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
#         ),
#         moderations=ModerationParameters(
#             hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
#         ),
#         data={"meal": "Lasagne"},
#     ),
#     lambda session_id: message_history,
#     input_messages_key="input",
#     history_messages_key="chat_history",
# )

# # Start a conversation with memory functionality
# print(heading("Start conversation with langchain"))
# prompt = "Describe what is Python in one sentence."
# print(f"Request: {prompt}")
# result = agent_with_chat_history.invoke(
#     [
#         SystemMessage(
#             content="""You are a helpful, respectful and honest assistant.
# Always answer as helpfully as possible, while being safe.
# Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
# Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
# any sense, or is not factually coherent, explain why instead of answering something incorrectly.
# If you don't know the answer to a question, please don't share false information.
# """,
#             user_id="assistant"
#         ),
#         HumanMessage(content=prompt, user_id="user")
#     ],
#     {"configurable": {"session_id": session_id}}
# )

# conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
# print(f"New conversation with ID '{conversation_id}' has been created!")
# print(f"Response: {result.generations[0][0].text}")
# print(result.llm_output)
# print(result.generations[0][0].generation_info)

# # Continue the conversation with memory functionality
# print(heading("Continue conversation with langchain"))
# prompt = "Show me some simple code example."
# print(f"Request: {prompt}")
# result = agent_with_chat_history.invoke(
#     {"input": [HumanMessage(content=prompt, user_id="user")]},
#     {"configurable": {"session_id": conversation_id}},
#     use_conversation_parameters=True,
# )
# print(f"Response: {result.generations[0][0].text}")

# # Delete the prompt if no longer needed
# client.prompt.delete(prompt_response.result.id)

## Langchain Agent using IBM SDK

In [31]:
from typing import Optional

from dotenv import load_dotenv
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import render_text_description_and_args
from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import BaseTool

from genai import Client, Credentials
from genai.extensions.langchain import LangChainChatInterface
from genai.schema import TextGenerationParameters

load_dotenv()


class WordLengthTool(BaseTool):
    name = "GetWordLength"
    description = "Returns the length of a word."

    def _run(self, word: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> int:
        return len(word)


tools: list[BaseTool] = [WordLengthTool()]

system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""
human_prompt = """{input}
{agent_scratchpad}
(reminder to respond in a JSON blob no matter what)"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

client = Client(credentials=Credentials.from_env())
llm = LangChainChatInterface(
    client=client,
    model_id="meta-llama/llama-2-70b-chat",
    parameters=TextGenerationParameters(
        max_new_tokens=250, min_new_tokens=20, temperature=0, stop_sequences=["\nObservation"]
    ),
)
prompt = prompt.partial(
    # get tools with their descriptions and args in plain text
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

memory = ConversationBufferMemory()

agent = (
    RunnablePassthrough.assign(
        # format the agent's scratchpad to a string
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        # pass the memory as the chat history
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory)

agent_executor.invoke({"input": "How many letters are in the word educa?"})
agent_executor.invoke({"input": "That's not a real word, can you tell me a valid word?"})
agent_executor.invoke({"input":"Can you suggest me some synonyms of the word education?"})




> Entering new AgentExecutor chain...
  Action:
```
{
  "action": "GetWordLength",
  "action_input": "educA"
}
```
Observation5  Action:
```
{
  "action": "Final Answer",
  "action_input": "The word 'educA' has 5 letters."
}
```
Observation

> Finished chain.


> Entering new AgentExecutor chain...
  {
"action": "GetWordLength",
"action_input": "educA"
}

Observation5  Action:
```
{
  "action": "GetWordLength",
  "action_input": "education"
}
```
Observation9  Action:
```
{
  "action": "Final Answer",
  "action_input": "The word 'education' has 9 letters."
}
```
Observation

> Finished chain.


> Entering new AgentExecutor chain...
  {
"action": "GetWordLength",
"action_input": "education"
}

{
"action": "GetWordLength",
"action_input": "learning"
}

{
"action": "GetWordLength",
"action_input": "instruction"
}

{
"action": "GetWordLength",
"action_input": "teaching"
}

{
"action": "GetWordLength",
"action_input": "schooling"
}

(reminder to respond directly)

Thought: The human is as

{'input': 'Can you suggest me some synonyms of the word education?',
 'history': "Human: How many letters are in the word educa?\nAI: The word 'educA' has 5 letters.\nHuman: That's not a real word, can you tell me a valid word?\nAI: The word 'education' has 9 letters.",
 'output': "Here are some synonyms of the word 'education': learning, instruction, teaching, schooling."}

## Chat a model using Langchain

In [32]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)

# make sure you have a .env file under genai root with
# GENAI_KEY=<your-genai-key>
# GENAI_API=<genai-api-endpoint> (optional) DEFAULT_API = "https://bam-api.res.ibm.com"
load_dotenv()


def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"


llm = LangChainChatInterface(
    client=Client(credentials=Credentials.from_env()),
    model_id="meta-llama/llama-2-70b-chat",
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=100,
        min_new_tokens=10,
        temperature=0.5,
        top_k=50,
        top_p=1,
        return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
    ),
    moderations=ModerationParameters(
        # Threshold is set to very low level to flag everything (testing purposes)
        # or set to True to enable HAP with default settings
        hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
    ),
)

print(heading("Start conversation with langchain"))
prompt = "Describe what is Python in one sentence."
print(f"Request: {prompt}")
result = llm.generate(
    messages=[
        [
            SystemMessage(
                content="""You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
any sense, or is not factually coherent, explain why instead of answering something incorrectly.
If you don't know the answer to a question, please don't share false information.
""",
            ),
            HumanMessage(content=prompt),
        ]
    ],
)
conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
print(f"New conversation with ID '{conversation_id}' has been created!")
print(f"Response: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

print(heading("Continue conversation with langchain"))
prompt = "Show me the example of code which can print fibonacci series."
print(f"Request: {prompt}")
result = llm.generate(
    messages=[[HumanMessage(content=prompt)]], conversation_id=conversation_id, use_conversation_parameters=True
)
# prompt = "Give me the fifth term for n =9."
# print(f"Request: {prompt}")
# result = llm.generate(
#     messages=[[HumanMessage(content=prompt)]], conversation_id=conversation_id, use_conversation_parameters=True
# )
print(f"Response: {result.generations[0][0].text}")


====================== Start conversation with langchain =======================

Request: Describe what is Python in one sentence.
New conversation with ID 'c82f8ef3-9710-4a86-9772-611d6b4d89a4' has been created!
Response:   Python is a high-level, interpreted programming language that is widely used for various purposes such as web development, data analysis, artificial intelligence, and scientific computing, known for its simplicity, readability, and ease of use.
{'model_name': 'meta-llama/llama-2-70b-chat', 'token_usage': {'prompt_tokens': 150, 'completion_tokens': 47, 'total_tokens': 197}}
{'meta': {'conversation_id': 'c82f8ef3-9710-4a86-9772-611d6b4d89a4', 'created_at': datetime.datetime(2024, 5, 7, 2, 44, 6, 545000, tzinfo=TzInfo(UTC)), 'id': '088cc289-9550-4c3d-8e22-ec8cc4d817cd', 'model_id': 'meta-llama/llama-2-70b-chat'}, 'token_usage': {'prompt_tokens': 150, 'completion_tokens': 47, 'total_tokens': 197}, 'generated_token_count': 47, 'input_token_count': 150, 'input_tokens':

#### Chat a model using langchain- IBM SDK - changed

In [13]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)

# Load environment variables
load_dotenv()

def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"

llm = LangChainChatInterface(
    client=Client(credentials=Credentials.from_env()),
    model_id="meta-llama/llama-2-70b-chat",
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=100,
        min_new_tokens=10,
        temperature=0.5,
        top_k=50,
        top_p=1,
        return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
    ),
    moderations=ModerationParameters(
        hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
    ),
)

print(heading("Start conversation with langchain"))
user_input = input("Enter your question: ")  # Prompt the user for input
result = llm.generate(
    messages=[
        [
            SystemMessage(
                content="""You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
any sense, or is not factually coherent, explain why instead of answering something incorrectly.
If you don't know the answer to a question, please don't share false information.
""",
            ),
            HumanMessage(content=user_input),
        ]
    ],
)
conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
print(f"New conversation with ID '{conversation_id}' has been created!")
print(f"Response: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

print(heading("Continue conversation with langchain"))
user_input = input("Enter your question: ")  # Prompt the user for input
result = llm.generate(
    messages=[[HumanMessage(content=user_input)]], conversation_id=conversation_id, use_conversation_parameters=True
)
print(f"Response: {result.generations[0][0].text}")


====================== Start conversation with langchain =======================



In [13]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# Placeholder for retriever - Define or import the retriever module here
retriever = None

# Load environment variables
load_dotenv()

def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"

def langchain_interaction(msg: dict, config: dict):
    try:
        return agent_with_chat_history.invoke(msg, config)
    except Exception as e:
        # Handle the exception here
        print(f"An error occurred: {e}")
        # You can log the error, retry, or take other actions as needed

# Create the custom RunnableLambda for LangChain interactions
langchain_error_handler = RunnableLambda(langchain_interaction)

# Create the custom template prompt
client = Client(credentials=Credentials.from_env())
prompt_response = client.prompt.create(
    name="Recipe Generator Prompt",
    model_id="google/flan-t5-xl",
    input="Make a short recipe for {{meal}} (use bullet points)",
)

# Create the LangChain Chat Interface with memory functionality
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    LangChainChatInterface(
        client=client,
        model_id="ibm/granite-13b-instruct-v2",
        prompt_id=prompt_response.result.id,
        parameters=TextGenerationParameters(
            decoding_method=DecodingMethod.SAMPLE,
            max_new_tokens=100,
            min_new_tokens=10,
            temperature=0.5,
            top_k=50,
            top_p=1,
            return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
        ),
        moderations=ModerationParameters(
            hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
        ),
        data={"meal": "Lasagne"},
    ),
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# def memory_runnable(x):
#     # Implement the logic to handle the chat history and other data
#     return {"ability": x["ability"], "chat_history": x["chat_history"], "question": x["question"]}
def memory_runnable(x):
    ability = x.get("ability")
    question = x.get("question")
    chat_history = x.get("chat_history", [])  # Default to an empty list if 'chat_history' key is not present
    return {"ability": ability, "chat_history": chat_history, "question": question}

# Add the setup_and_retrieval code block
setup_and_retrieval = RunnableParallel(
    {
        "response_from_vector_store": itemgetter("question") | (retriever if retriever else RunnablePassthrough()),
        "response_to_chat_history": RunnableLambda(memory_runnable) | memory_runnable,
        "question": RunnablePassthrough.assign(question=itemgetter("question")),
        "ability": RunnablePassthrough.assign(ability=itemgetter("ability")),
    }
)

# Use the RunnableLambda in the LangChain workflow
# langchain_workflow = langchain_error_handler.chain(setup_and_retrieval, agent_with_chat_history)
langchain_workflow = RunnableParallel(
    {
        "langchain_error_handler": langchain_error_handler,
        "setup_and_retrieval": setup_and_retrieval,
        "agent_with_chat_history": agent_with_chat_history
    }
)

# Start the conversation with memory functionality
print(heading("Start conversation with langchain"))
prompt = "Describe what is Python in one sentence."
print(f"Request: {prompt}")
result = langchain_workflow.invoke(
    {
        "input": [
            SystemMessage(
                content="""You are a helpful, respectful and honest assistant.
                        Always answer as helpfully as possible, while being safe.
                        Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
                        Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
                        any sense, or is not factually coherent, explain why instead of answering something incorrectly.
                        If you don't know the answer to a question, please don't share false information.""",
                user_id="assistant"
            ),
            HumanMessage(content=prompt, user_id="user")
        ],
        "ability": "python-description",
        "question": prompt,
    },
    config = {"configurable": {"session_id": session_id}}
)

conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
print(f"New conversation with ID '{conversation_id}' has been created!")
print(f"Response: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

# Continue the conversation with memory functionality
print(heading("Continue conversation with langchain"))
prompt = "Show me some simple code example."
print(f"Request: {prompt}")
result = langchain_workflow.invoke(
    {
        "input": [HumanMessage(content=prompt, user_id="user")],
        "ability": "code-example",
        "question": prompt,
    },
    {"configurable": {"session_id": conversation_id}},
    use_conversation_parameters=True,
)
print(f"Response: {result.generations[0][0].text}")

# Delete the prompt if no longer needed
client.prompt.delete(prompt_response.result.id)


====================== Start conversation with langchain =======================

Request: Describe what is Python in one sentence.
An error occurred: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.


ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [39]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# Placeholder for retriever - Define or import the retriever module here
retriever = None

# Load environment variables
load_dotenv()

def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"

def langchain_interaction(msg: dict, config: dict):
    try:
        return agent_with_chat_history.invoke(msg, config)
    except Exception as e:
        # Handle the exception here
        print(f"An error occurred: {e}")
        # You can log the error, retry, or take other actions as needed

# Create the custom RunnableLambda for LangChain interactions
langchain_error_handler = RunnableLambda(langchain_interaction)

# Create the custom template prompt
client = Client(credentials=Credentials.from_env())
prompt_response = client.prompt.create(
    name="Recipe Generator Prompt",
    model_id="google/flan-t5-xl",
    input="Make a short recipe for {{meal}} (use bullet points)",
)

# Create the LangChain Chat Interface with memory functionality
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    LangChainChatInterface(
        client=client,
        model_id="ibm/granite-13b-instruct-v2",
        prompt_id=prompt_response.result.id,
        parameters=TextGenerationParameters(
            decoding_method=DecodingMethod.SAMPLE,
            max_new_tokens=100,
            min_new_tokens=10,
            temperature=0.5,
            top_k=50,
            top_p=1,
            return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
        ),
        moderations=ModerationParameters(
            hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
        ),
        data={"meal": "Lasagne"},
    ),
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# def memory_runnable(x):
#     # Implement the logic to handle the chat history and other data
#     return {"ability": x["ability"], "chat_history": x["chat_history"], "question": x["question"]}
def memory_runnable(x):
    ability = x.get("ability")
    question = x.get("question")
    chat_history = x.get("chat_history", [])  # Default to an empty list if 'chat_history' key is not present
    return {"ability": ability, "chat_history": chat_history, "question": question}

# Add the setup_and_retrieval code block
setup_and_retrieval = RunnableParallel(
    {
        "response_from_vector_store": itemgetter("question") | (retriever if retriever else RunnablePassthrough()),
        "response_to_chat_history": RunnableLambda(memory_runnable) | memory_runnable,
        "question": RunnablePassthrough.assign(question=itemgetter("question")),
        "ability": RunnablePassthrough.assign(ability=itemgetter("ability")),
    }
)

# Use the RunnableLambda in the LangChain workflow
# langchain_workflow = langchain_error_handler.chain(setup_and_retrieval, agent_with_chat_history)
langchain_workflow = RunnableParallel(
    {
        "langchain_error_handler": langchain_error_handler,
        "setup_and_retrieval": setup_and_retrieval,
        "agent_with_chat_history": agent_with_chat_history
    }
)

# Start the conversation with memory functionality
print(heading("Start conversation with langchain"))
prompt = "Describe what is Python in one sentence."
print(f"Request: {prompt}")
# input_messages = [
#     SystemMessage(content=""""You are a helpful, respectful and honest assistant.
#                         Always answer as helpfully as possible, while being safe.
#                         Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
#                         Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
#                         any sense, or is not factually coherent, explain why instead of answering something incorrectly.
#                         If you don't know the answer to a question, please don't share false information.
#                             """ , user_id="assistant"),
#     HumanMessage(content=prompt, user_id="user")
# ]

# result = langchain_workflow.invoke(
#     {
#         "input": input_messages,
#         "ability": "python-description",
#         "question": prompt,
#     },
#     config = {"configurable": {"session_id": session_id}}
# )

conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
print(f"New conversation with ID '{conversation_id}' has been created!")
print(f"Response: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

# Continue the conversation with memory functionality
print(heading("Continue conversation with langchain"))
prompt = "Show me some simple code example."
print(f"Request: {prompt}")
result = langchain_workflow.invoke(
    {
        # "input": HumanMessage(content=prompt, user_id="user"),
        "input":"Show me some simple code example",
        "ability": "code-example",
        "question": "Show me some simple code example",
    },
    {"configurable": {"session_id": conversation_id}},
    # use_conversation_parameters=True
)
print(f"Response: {result.generations[0][0].text}")

# Delete the prompt if no longer needed
client.prompt.delete(prompt_response.result.id)


====================== Start conversation with langchain =======================

Request: Describe what is Python in one sentence.
New conversation with ID 'c82f8ef3-9710-4a86-9772-611d6b4d89a4' has been created!
Response:   Sure! Here's an example of Python code that prints the Fibonacci series up to a given number:
```
def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        print(b, end=" ")
        a, b = b, a + b
    print()

# Example usage:
fibonacci(5)
```
This code defines a
{'model_name': 'meta-llama/llama-2-70b-chat', 'token_usage': {'prompt_tokens': 218, 'completion_tokens': 100, 'total_tokens': 318}}
{'meta': {'conversation_id': 'c82f8ef3-9710-4a86-9772-611d6b4d89a4', 'created_at': datetime.datetime(2024, 5, 7, 2, 44, 11, 486000, tzinfo=TzInfo(UTC)), 'id': 'd4a2e560-93eb-4d0f-9a14-00fad9a231db', 'model_id': 'meta-llama/llama-2-70b-chat'}, 'token_usage': {'prompt_tokens': 218, 'completion_tokens': 100, 'total_tokens': 318}, 'generated_token_count': 100, 'input_tok

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

#### Changed code

In [75]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Placeholder for retriever - Define or import the retriever module here
retriever = None

# Load environment variables
load_dotenv()

def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"

def langchain_interaction(msg: dict, config: dict):
    try:
        return agent_with_chat_history.invoke(msg, config)
    except Exception as e:
        # Handle the exception here
        print(f"An error occurred: {e}")
        # You can log the error, retry, or take other actions as needed

# Create the custom RunnableLambda for LangChain interactions
langchain_error_handler = RunnableLambda(langchain_interaction)

# Create the custom template prompt
client = Client(credentials=Credentials.from_env())
prompt_response = client.prompt.create(
    name="Recipe Generator Prompt",
    model_id="google/flan-t5-xl",
    input="Make a short recipe for {{meal}} (use bullet points)",
)

# Create the LangChain Chat Interface with memory functionality
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    LangChainChatInterface(
        client=client,
        model_id="ibm/granite-13b-instruct-v2",
        prompt_id=prompt_response.result.id,
        parameters=TextGenerationParameters(
            decoding_method=DecodingMethod.SAMPLE,
            max_new_tokens=100,
            min_new_tokens=10,
            temperature=0.5,
            top_k=50,
            top_p=1,
            return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
        ),
        moderations=ModerationParameters(
            hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
        ),
        data={"meal": "Lasagne"},
    ),
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Define the setup_and_retrieval workflow
# setup_and_retrieval = RunnableParallel(
#     {
#         "response_from_vector_store": itemgetter("question") | (retriever if retriever else RunnablePassthrough()),
#         "response_to_chat_history": RunnableLambda(lambda x: x) | RunnablePassthrough(),
#         "question": RunnablePassthrough.assign(question=itemgetter("question")),
#         "ability": RunnablePassthrough.assign(ability=itemgetter("ability")),
#     }
# )
# Define the setup_and_retrieval workflow
# setup_and_retrieval = RunnableParallel(
#     {
#         "response_from_vector_store": itemgetter("question") | (retriever if retriever else RunnablePassthrough()),
#         "response_to_chat_history": RunnableLambda(lambda x: x) | RunnablePassthrough(),
#         "question": RunnablePassthrough.assign(question=lambda prompt_value: prompt_value.to_dict()["question"]),
#         "ability": RunnablePassthrough.assign(ability=lambda prompt_value: prompt_value.to_dict()["ability"]),
#     }
# )
# Define the setup_and_retrieval workflow
setup_and_retrieval = RunnableParallel(
    {
        "response_from_vector_store": itemgetter("question") | (retriever if retriever else RunnablePassthrough()),
        "response_to_chat_history": RunnableLambda(lambda x: x) | RunnablePassthrough(),
        "question": lambda prompt_value: prompt_value.to_dict()["question"],
        "ability": lambda prompt_value: prompt_value.to_dict()["ability"],
    }
)
# Define the LangChain workflow
langchain_workflow = RunnableParallel(
    {
        "langchain_error_handler": langchain_error_handler,
        "setup_and_retrieval": setup_and_retrieval,
        "agent_with_chat_history": agent_with_chat_history
    }
)

# Start the conversation with memory functionality
print(heading("Start conversation with langchain"))
prompt = "Describe what is Python in one sentence."
print(f"Request: {prompt}")
input_message = HumanMessage(content=prompt, user_id="user")
# class PromptValue:
#     def __init__(self, input_messages, ability, question):
#         self.input_messages = input_messages
#         self.ability = ability
#         self.question = question
class PromptValue:
    def __init__(self, input_messages, ability, question):
        self.input = input_messages
        self.ability = ability
        self.question = question

    def __getitem__(self, key):
        if key == "input":
            return self.input
        elif key == "ability":
            return self.ability
        elif key == "question":
            return self.question
        else:
            raise KeyError(f"Key '{key}' not found in PromptValue object.")

    def __iter__(self):
        return iter(["input", "ability", "question"])

    def to_dict(self):
        return {
            "input": self.input,
            "ability": self.ability,
            "question": self.question
        }
prompt_value = PromptValue(input_messages=[input_message], ability="python-description", question=prompt)
result = langchain_workflow.invoke(prompt_value, config={"configurable": {"session_id": "<foo>"}})
# result = langchain_workflow.invoke(
#     {
#         "input": [HumanMessage(content=prompt, user_id="user")],
#         "ability": "python-description",
#         "question": prompt,
#     },
#     config={"configurable": {"session_id": "<foo>"}}
# )

conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
print(f"New conversation with ID '{conversation_id}' has been created!")
print(f"Response: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

# Continue the conversation with memory functionality
print(heading("Continue conversation with langchain"))
prompt = "Show me some simple code example."
print(f"Request: {prompt}")

result = langchain_workflow.invoke(
    {
        "input": [HumanMessage(content=prompt, user_id="user")],
        "ability": "code-example",
        "question": prompt,
    },
    config={"configurable": {"session_id": conversation_id}}
)

print(f"Response: {result.generations[0][0].text}")

# Delete the prompt if no longer needed
client.prompt.delete(prompt_response.result.id)


====================== Start conversation with langchain =======================

Request: Describe what is Python in one sentence.
An error occurred: The input to RunnablePassthrough.assign() must be a dict.


AssertionError: The input to RunnablePassthrough.assign() must be a dict.

In [72]:
prompt_value.question

'Describe what is Python in one sentence.'

In [64]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools import Tool
from langchain import hub
import typing
from langchain.prompts import PromptTemplate

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

class ChatPromptTemplateConverter:
    @staticmethod
    def convert(chat_prompt_template):
        input_variables = chat_prompt_template.input_variables
        input_types = chat_prompt_template.input_types
        metadata = chat_prompt_template.metadata
        messages = [message.to_string() for message in chat_prompt_template.messages]

        return PromptTemplate(input_variables=input_variables, input_types=input_types, metadata=metadata, messages=messages)

# Assuming you have the 'prompt' object from hub.pull("hwchase17/openai-functions-agent")
prompt = ChatPromptTemplateConverter.convert(prompt)

# Define your custom tools
def my_tool(query: str) -> str:
    """A custom tool that can be used by the agent."""
    return f"This is the result of using the '{query}' tool."

tool = Tool(
    name="my_tool",
    func=my_tool,
    description="A custom tool that can be used by the agent."
)

# Create the agent
llm = "ibm/granite-13b-instruct-v2"
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run the agent
result = agent_executor.run("Use the my_tool tool to do something.")
print(result)


AttributeError: 'SystemMessagePromptTemplate' object has no attribute 'to_string'

In [61]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

## Using chatprompt template

In [9]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Placeholder for retriever - Define or import the retriever module here
retriever = None

# Load environment variables
load_dotenv()

def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"

def langchain_interaction(msg: dict, config: dict):
    try:
        return agent_with_chat_history.invoke(msg, config)
    except Exception as e:
        # Handle the exception here
        print(f"An error occurred: {e}")

# Create the custom RunnableLambda for LangChain interactions
langchain_error_handler = RunnableLambda(langchain_interaction)

# Create the custom template prompt
client = Client(credentials=Credentials.from_env())
prompt_response = client.prompt.create(
    name="Recipe Generator Prompt",
    model_id="google/flan-t5-xl",
    input="Make a short recipe for {{meal}} (use bullet points)",
)

# Create the LangChain Chat Interface with memory functionality
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    LangChainChatInterface(
        client=client,
        model_id="ibm/granite-13b-instruct-v2",
        prompt_id=prompt_response.result.id,
        parameters=TextGenerationParameters(
            decoding_method=DecodingMethod.SAMPLE,
            max_new_tokens=100,
            min_new_tokens=10,
            temperature=0.5,
            top_k=50,
            top_p=1,
            return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
        ),
        moderations=ModerationParameters(
            hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
        ),
        data={"meal": "Lasagne"},
    ),
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Create the ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate(
        message=SystemMessage(
            content="""Thank you for engaging with me! I strive to provide accurate and helpful responses while maintaining a safe and respectful environment.
            Your questions are valuable, and I'm here to assist you to the best of my abilities. Let's explore and learn together!""",
            user_id="assistant"
        )
    ),
    HumanMessagePromptTemplate(
        message=HumanMessage(content="{query}", user_id="user")
    )
])

# Define langchain_workflow
langchain_workflow = RunnableParallel(
    {
        "langchain_error_handler": langchain_error_handler,
        "agent_with_chat_history": agent_with_chat_history
    }
)

# Start the conversation with memory functionality using ChatPromptTemplate
print(heading("Start conversation with langchain"))
prompt = "Describe what is Python in one sentence."
print(f"Request: {prompt}")
result = langchain_workflow.invoke(
    chat_prompt.format_prompt(query=prompt).dict(),
    {"configurable": {"session_id": session_id}}
)

# Start the conversation with memory functionality using RunnablePassthrough
# print(heading("Start conversation with langchain"))
# prompt = "Describe what is Python in one sentence."
# print(f"Request: {prompt}")
# result = langchain_workflow.invoke(
#     RunnablePassthrough.assign(
#         input=[
#             SystemMessage(
#                 content="""Thank you for engaging with me! I strive to provide accurate and helpful responses while maintaining a safe and respectful environment.
#                 Your questions are valuable, and I'm here to assist you to the best of my abilities. Let's explore and learn together!""",
#                 user_id="assistant"
#             ),
#             HumanMessage(content=prompt, user_id="user")
#         ],
#         ability="python-description",
#         question=prompt
#     ),
#     {"configurable": {"session_id": session_id}}
# )

# Continue the conversation with memory functionality
print(heading("Continue conversation with langchain"))
prompt = "Show me some simple code example."
print(f"Request: {prompt}")
result = langchain_workflow.invoke(
    {
        "input": [HumanMessage(content=prompt, user_id="user")],
        "ability": "code-example",
        "question": prompt,
    },
    {"configurable": {"session_id": conversation_id}},
    use_conversation_parameters=True,
)

# Delete the prompt if no longer needed
client.prompt.delete(prompt_response.result.id)

ValidationError: 1 validation error for SystemMessagePromptTemplate
prompt
  field required (type=value_error.missing)